<a href="https://colab.research.google.com/github/Dilraj0/comp215/blob/main/labs/lab02-review.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

COMP 215 - LAB 2
----------------
#### Name(s):
#### Date:

By the end of this lab you should be able to:
  * use strings, tuples, lists, and dictionaries (review)
  * use *f-string* to simplify string formatting operations
  * write functions and simple unit tests (review)
  * use *list comprehension* to implement map and filter algorithms


During this lab, you will be introduced to the following:
  * API Keys
  * JSON data formats
  * the `datetime` module to represent dates and times
  * the `pprint` module to format data structures

## Near Earth Objects

In this lab, we'll answer some questions about [Near Earth Objects](https://cneos.jpl.nasa.gov/) using NASA's API:  [https://api.nasa.gov/](https://api.nasa.gov/#NeoWS).  But, as usual, let's do the imports first.


In [ ]:
import datetime, json, requests
from pprint import pprint    # Pretty Print - built-in python function to nicely format data structures

### dates & f-strings

First we need a short tutorial on python dates and [f-strings](https://realpython.com/python-f-strings/)...

In [ ]:
today = datetime.date.today()   # get a date object representing today's date
print(today, type(today))
formatted_date = f'Today is: {today}'   # A format string - notice how the variable `today` is formatted into the string
print(formatted_date)

2025-01-15 <class 'datetime.date'>
Today is: 2025-01-15


### Make an API Query

Let's get some data from the NEO database; here's a query that gets the observation "feed" for today.  Notice the first line of the block is an 'API key'.  An API key is a unique identifier that authenticates a user when making a request to an API.  Run the block below using the API key 'DEMO_KEY' (just to be sure it works), then go to https://api.nasa.gov and get your own API key.  Use your own API key for the remainder of the lab.

In [13]:
API_KEY = 'DEMO_KEY'  # substitute your API key here

# get today's date (as a string)
#today = str(datetime.date.today())
today = '2023-01-09'

# use an f-string to "format" the date and API key varaibles.
url = f'https://api.nasa.gov/neo/rest/v1/feed?start_date={today}&end_date={today}&api_key={API_KEY}'

# make the API request (recall from lab 1)
response = requests.request("GET", url, headers={}, data={})

# convert the response to json format (recall from lab 1)
data = json.loads(response.text)

# TIP: print(data) to see the whole data structure returned, here we grab just the list of NEO's:
n_results = data['element_count']
neos = data['near_earth_objects'][today]
print(f'{n_results} Near Earth Objects found for {today}')
pprint(neos[:1])

14 Near Earth Objects found for 2023-01-09
[{'absolute_magnitude_h': 19.62,
  'close_approach_data': [{'close_approach_date': '2023-01-09',
                           'close_approach_date_full': '2023-Jan-09 13:59',
                           'epoch_date_close_approach': 1673272740000,
                           'miss_distance': {'astronomical': '0.0725378623',
                                             'kilometers': '10851509.694433301',
                                             'lunar': '28.2172284347',
                                             'miles': '6742815.4556135138'},
                           'orbiting_body': 'Earth',
                           'relative_velocity': {'kilometers_per_hour': '32387.3479203461',
                                                 'kilometers_per_second': '8.9964855334',
                                                 'miles_per_hour': '20124.2384897284'}}],
  'estimated_diameter': {'feet': {'estimated_diameter_max': 2322.8706681664,
     

Next we extract just the potentially hazerdous asteroids, using a Comp115-style list accumulator *loop*:

In [14]:
hazards =  []
for item in neos:
  if item['is_potentially_hazardous_asteroid'] is True:
    hazards.append(item)
print(f'{len(hazards)} potentially hazardous asteroids identified.')

3 potentially hazardous asteroids identified.


### Fetch Complete Data for One Asteroid

Notice that the record for each `neo` is a dictionary with `id` field that uniquely identifies this record in the database.  We can use this `id` to fetch complete orbital and close approach data for the NEO.  For example, this query fetches the complete data set for the first hazardous asteroid...  Notice that the `miss_distance` field contains the distance (in various units) by which the NEO missed an "orbiting body".


In [6]:
id = hazards[0]['id']
url = f'https://api.nasa.gov/neo/rest/v1/neo/{id}?api_key={API_KEY}'
response = requests.request("GET", url, headers={}, data={})
data = json.loads(response.text)

pprint(data)

NameError: name 'hazards' is not defined

## Exercise 1

In the code cell below, **re-write the accumulator loop** that creates the list of hazards as a [list comprehension](https://realpython.com/lessons/list-comprehensions-overview/) that implements a ["filter"](https://youtu.be/hUes6y2b--0)
Notice how this provides a concise way to "filter" items of interest from a larger data set.

In [21]:
import datetime, json, requests
from pprint import pprint
API_KEY = 'DEMO_KEY'
today = str(datetime.date.today())
url = f'https://api.nasa.gov/neo/rest/v1/feed?start_date={today}&end_date={today}&api_key={API_KEY}'
response = requests.get(url)
data = response.json()
neos = data['near_earth_objects'][today]

# Rewrite the accumulator loop as a list comprehension
hazards = [item for item in neos if item['is_potentially_hazardous_asteroid']]
print(f'{len(hazards)} potentially hazardous asteroids identified.')

KeyError: 'near_earth_objects'



## Exercise 2

In the code cell below, write a python function that takes a list of "close approach data" as a parameter,
and returns a 2-tuple with the (date, miss km) of the closest approach to Earth in the list (where "miss km" is the miss distance in km).

Hints:
* notice the input is a list of dictionaries.  Each dictionary has a 'close_approach_date", "orbiting_body", and 'miss_distance' field.
* we are only interested in the closest approach to "Earth"


Add at least one unit test to check your work - note the test data only needs dictionaries with the fields your function actually uses.


In [16]:
def closest_approach(close_approach_data):
    # Filter for Earth approaches and find the minimum distance
    earth_approaches = [
        (approach.get('close_approach_date', approach.get('close_approach_date_full')),  # Use get to handle missing keys
         float(approach['miss_distance']['kilometers']))
        for approach in close_approach_data
        if approach['orbiting_body'] == 'Earth'
    ]
    return min(earth_approaches, key=lambda x: x[1])

# Unit test for Exercise 2
test_data = [
    {

         'close_approach_date_full': '2023-Jan-09 13:59',
                           'epoch_date_close_approach': 1673272740000,
                           'miss_distance': {'astronomical': '0.0725378623',
                                             'kilometers': '10851509.694433301',
                                             'lunar': '28.2172284347',
                                             'miles': '6742815.4556135138'},
                           'orbiting_body': 'Earth',
                           'relative_velocity': {'kilometers_per_hour': '32387.3479203461',
                                                 'kilometers_per_second': '8.9964855334',
                                                 'miles_per_hour': '20124.2384897284'}},
    {
        'close_approach_date': '2023-01-10',
        'orbiting_body': 'Earth',
        'miss_distance': {'kilometers': '5051009.694433301'}
    }
]

result = closest_approach(test_data)
print(f"Closest approach: {result}")  # Expected: ('2023-01-10', 5051009.694433301)

Closest approach: ('2023-01-10', 5051009.694433301)


## Challenge Exercise

In the code cell below, write a complete program that:
 1. fetches the list of NEO's for this week.
 2. for each NEO, fetch it's complete orbital data and determine its closest approach to Earth
 3. identify which NEO from this week's data makes the closet approach to earth
 4. print a nice message with information about the NEO, which it will approach the Earth, and how close it will come.

Hints:
* you'll need the start and end date - end date is today, see if you can use a [`timedelta`](https://docs.python.org/3/library/datetime.html#timedelta-objects) object to ge the start date (you can do basic "date math" with `timedelta` and `date` objects!)
* you may need to modify the function we wrote in Ex. 2 to return a triple with the NEO's id included;
* lots of opportunity here for more practice with list comprehensions


In [20]:
import datetime
import requests

API_KEY = 'DEMO_KEY'

# Calculate the start and end dates for this week
today = datetime.date.today()
start_date = today - datetime.timedelta(days=7)
end_date = today

# Fetch NEO data for the week
url = f'https://api.nasa.gov/neo/rest/v1/feed?start_date={start_date}&end_date={end_date}&api_key={API_KEY}'
response = requests.get(url)

if response.status_code == 200:  # Check if the API call was successful
    data = response.json()

    # Initialize variables to track the closest NEO
    closest_neo = None
    closest_distance = float('inf')

    # Loop through the dates and NEOs
    for date in data['near_earth_objects']:
        for neo in data['near_earth_objects'][date]:
            # Loop through close approach data
            for approach in neo.get('close_approach_data', []):
                if approach.get('orbiting_body') == 'Earth':
                    distance = float(approach['miss_distance']['kilometers'])
                    if distance < closest_distance:
                        closest_distance = distance
                        closest_neo = {
                            'name': neo['name'],
                            'date': approach['close_approach_date'],
                            'distance': distance
                        }

    # Print the result
    if closest_neo:
        print(
            f"The closest NEO this week is {closest_neo['name']} on {closest_neo['date']} "
            f"at a distance of {closest_neo['distance']:.2f} km."
        )
    else:
        print("No close approaches to Earth this week.")
else:
    print(f"Failed to fetch data from NASA API. Status code: {response.status_code}")



Failed to fetch data from NASA API. Status code: 429
